# **Convolutional Neural Network Architectures for Image Analysis**

**Objectives:**
- Understand how to setup a convolutional neural network (CNN) through 5 steps: architecture setup, optimisation setup, training, validation and testing
- Understand the residual neural network (ResNet) architecture
- Understand the U-net architecture

CNN is a deep learning algorithm that takes an image as an input and has learnable weights and biases which learn filters which highlight features in the image (e.g. edges or objects) which is needed for model production.


A CNN is a deep learning algorithm designed to process and analyse visual data, such as images. It automatically learns patterns and features from an image by applying filters with learnable weights and biases. These filters detect important features like edges, textures, shapes and eventaully entire objects. One of the key strengths of CNNs is their ability to recognise spatial hierarchies of features. This means the network learns to detect simple patterns (e.g edges) in the early layers, then combines these patterns to identify more complex structures (e.g. corners and textures) in deeper layers, and finally recognises entire objects(e.g. faces, cars). This step-by-step feature extraction allows CNN's to understand images in a way that mimics of humans percieve what we see. CNNs can automatically learn which features are important, making them highly effective for tasks such as **image classification**, **facial recognition** and **object detection**.

